In [1]:
import pandas as pd
import numpy as np
import re
import gdown

In [2]:
# enron_data = pd.read_csv('../data/emails.csv', quotechar='"', on_bad_lines='warn', engine='python')
try:
  # https://drive.google.com/file/d/17h6zOKETcEIdrcTQipPEGv1oTmOE3CB1/view?usp=sharing
  gdown.download('https://drive.google.com/uc?id=17h6zOKETcEIdrcTQipPEGv1oTmOE3CB1', 'emails.csv', quiet=False)
  # enron_data = pd.read_csv('../data/emails.csv', quotechar='"', on_bad_lines='warn')
  enron_data = pd.read_csv('emails.csv', quotechar='"', on_bad_lines='warn')
  # enron_data.head(5)
  print("File loaded successfully with quotechar and escapechar!")
except pd.errors.ParserError as e:
    print(f"Error loading file with quotechar and escapechar: {e}")

enron_data.head(5)

Downloading...
From (original): https://drive.google.com/uc?id=17h6zOKETcEIdrcTQipPEGv1oTmOE3CB1
From (redirected): https://drive.google.com/uc?id=17h6zOKETcEIdrcTQipPEGv1oTmOE3CB1&confirm=t&uuid=71276177-8e25-441c-a5ac-140a48bfabee
To: /Users/pkchoy/code/data_science_bootcamp/predictor/notebooks/emails.csv
 12%|█▏        | 172M/1.43G [00:09<01:07, 18.7MB/s] 

KeyboardInterrupt: 

 12%|█▏        | 173M/1.43G [00:20<01:07, 18.7MB/s]

In [ ]:
enron_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [ ]:
enron_data.iloc[::5000, 0]

0                      allen-p/_sent_mail/1.
5000              arnold-j/deleted_items/14.
10000                bass-e/_sent_mail/1051.
15000         bass-e/discussion_threads/827.
20000     baughman-d/power/cinergy_index/46.
                         ...                
495000              watson-k/e_mail_bin/584.
500000                 whalley-g/markets/20.
505000            white-s/deleted_items/967.
510000                   williams-w3/bill/8.
515000                   ybarbo-p/inbox/616.
Name: file, Length: 104, dtype: object

In [ ]:
# print(enron_data.iloc[5230, 1])
print(enron_data.iloc[5230, 1])
enron_data.iloc[5230, 1]

Message-ID: <9674676.1075852700004.JavaMail.evans@thyme>
Date: Thu, 18 Oct 2001 15:30:00 -0700 (PDT)
From: dutch.quigley@enron.com
To: ravi.thuraisingham@enron.com
Subject: RE: Neural Networks
Cc: john.arnold@enron.com
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
Bcc: john.arnold@enron.com
X-From: Quigley, Dutch </O=ENRON/OU=NA/CN=RECIPIENTS/CN=DQUIGLE>
X-To: Thuraisingham, Ravi </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Ravi_thuraisingham>
X-cc: Arnold, John </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Jarnold>
X-bcc: 
X-Folder: \JARNOLD (Non-Privileged)\Arnold, John\Deleted Items
X-Origin: Arnold-J
X-FileName: JARNOLD (Non-Privileged).pst

Here is the EOL information

 
Dutch

 -----Original Message-----
From: 	Arnold, John  
Sent:	Thursday, October 18, 2001 5:21 PM
To:	Quigley, Dutch
Subject:	FW: Neural Networks

This guy is working on an EOL project for me right now.  Can you send him a file with our EOL trades for today or yesterday with product, price, b

'Message-ID: <9674676.1075852700004.JavaMail.evans@thyme>\nDate: Thu, 18 Oct 2001 15:30:00 -0700 (PDT)\nFrom: dutch.quigley@enron.com\nTo: ravi.thuraisingham@enron.com\nSubject: RE: Neural Networks\nCc: john.arnold@enron.com\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nBcc: john.arnold@enron.com\nX-From: Quigley, Dutch </O=ENRON/OU=NA/CN=RECIPIENTS/CN=DQUIGLE>\nX-To: Thuraisingham, Ravi </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Ravi_thuraisingham>\nX-cc: Arnold, John </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Jarnold>\nX-bcc: \nX-Folder: \\JARNOLD (Non-Privileged)\\Arnold, John\\Deleted Items\nX-Origin: Arnold-J\nX-FileName: JARNOLD (Non-Privileged).pst\n\nHere is the EOL information\n\n \nDutch\n\n -----Original Message-----\nFrom: \tArnold, John  \nSent:\tThursday, October 18, 2001 5:21 PM\nTo:\tQuigley, Dutch\nSubject:\tFW: Neural Networks\n\nThis guy is working on an EOL project for me right now.  Can you send him a file with our EOL trades for toda

In [ ]:
def extract_email_primary_compo(email_msg):

    if not isinstance(email_msg, str):
        return [None] * 18

    pattern = re.compile(r"""
        ^Message-ID:\s+<(.*?)>\n  # Message ID (group 1)
        Date:\s+(.*?)\n           # Date (group 2)
        From:\s+(.*?)\n           # From (group 3)
        To:\s+(.*?)\n             # To (group 4)
        Subject:\s+(.*?)\n        # Subject (group 5)
        (?:Cc:\s+(.*?)\n)?        # Cc (group 6, optional)
        Mime-Version:\s+(.*?)\n   # Mime-Version (group 7)
        Content-Type:\s+(.*?)\n   # Content-Type (group 8)
        Content-Transfer-Encoding:\s+(.*?)\n  # Content-Transfer-Encoding (group 9)
        (?:Bcc:\s(.*?)\n)?        # Bcc (group 10, optional)
        X-From:\s+(.*?)\n         # X-From (group 11)
        X-To:\s+(.*?)\n           # X-To (group 12)
        X-cc:\s+(.*?)\n           # X-cc (group 13)
        X-bcc:\s+(.*?)\n          # X-bcc (group 14)
        X-Folder:\s+(.*?)\n       # X-Folder (group 15)
        X-Origin:\s+(.*?)\n       # X-Origin (group 16)
        X-FileName:\s+(.*?)\n\n   # X-FileName (group 17)
        (.*?)$                    # Message body (group 18)
        """, re.VERBOSE | re.DOTALL)

    # pattern = re.compile(r"""
    #     Date:\s+(.*?)\n           # Date (group 1)
    #     From:\s+(.*?)\n           # From (group 2)
    #     To:\s+(.*?)\n             # To (group 3)
    #     Subject:\s+(.*?)\n        # Subject (group 4)
    #     (?:Cc:\s+(.*?)\n)?             # Cc (group 5)
    #     (?:Bcc:\s(.*?)\n)?             # Bcc (group 6)
    #     X-From:\s+(.*?)\n         # X-From (group 7)
    #     X-To:\s+(.*?)\n           # X-To (group 8)
    #     X-cc:\s+(.*?)\n           # X-cc (group 9)
    #     X-bcc:\s+(.*?)\n          # X-bcc (group 10)
    #     (?:.*?\n)*?               # Skip unwanted metadata
    #     (?:\n\n|\n)
    #     (.*?)$                    # Message body (group 11)
    #     """, re.VERBOSE | re.DOTALL)

    match = pattern.search(email_msg)
    # print(match)

    if match:
        # print(match.group(1), match.group(2), match.group(3), match.group(4),
        #         match.group(5), match.group(6), match.group(7), match.group(8),
        #         match.group(9), match.group(10), match.group(11), match.group(12),
        #         match.group(13), match.group(14), match.group(15), match.group(16))

        return [match.group(1), match.group(2), match.group(3),
                match.group(4), match.group(5), match.group(6),
                match.group(7), match.group(8), match.group(9),
                match.group(10), match.group(11), match.group(12),
                match.group(13), match.group(14), match.group(15),
                match.group(16), match.group(17), match.group(18)]

    else:
        return [None] * 18


enron_primary_msg_series = enron_data['message'].apply(extract_email_primary_compo)
# print('whole series', enron_primary_msg_series)
# print('single cell', enron_primary_msg_series[10])
# print('shape', enron_primary_msg_series.shape)

# enron_primary_msg_list = enron_primary_msg_series.tolist()
# print('list: ', enron_primary_msg_list)

extracted_emails_df = pd.DataFrame(enron_primary_msg_series.tolist(),
                                                columns=['Message-ID', 'Date', 'From', 'To', 'Subject', 'Cc',
                                                          'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding',
                                                            "Bcc",'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin',
                                                              'X-FileName', 'Message-Body'])
# extracted_emails_df = pd.DataFrame(enron_primary_msg_series.tolist(),
#                                                 columns=['Date', 'From', 'To', 'Subject','Cc', 'Bcc',
#                                                             'X-From', 'X-To', 'X-cc', 'X-bcc', 'Message-Body'])

print('extracted: ', extracted_emails_df)

extracted_emails_df.to_csv('../data/01_extracted_emails.csv', index=False)

extracted:                                           Message-ID  \
0       18782981.1075855378110.JavaMail.evans@thyme   
1       15464986.1075855378456.JavaMail.evans@thyme   
2       24216240.1075855687451.JavaMail.evans@thyme   
3       13505866.1075863688222.JavaMail.evans@thyme   
4       30922949.1075863688243.JavaMail.evans@thyme   
...                                             ...   
517396  26807948.1075842029936.JavaMail.evans@thyme   
517397  25835861.1075842029959.JavaMail.evans@thyme   
517398  28979867.1075842029988.JavaMail.evans@thyme   
517399  22052556.1075842030013.JavaMail.evans@thyme   
517400  28618979.1075842030037.JavaMail.evans@thyme   

                                         Date                     From  \
0       Mon, 14 May 2001 16:39:00 -0700 (PDT)  phillip.allen@enron.com   
1        Fri, 4 May 2001 13:51:00 -0700 (PDT)  phillip.allen@enron.com   
2       Wed, 18 Oct 2000 03:00:00 -0700 (PDT)  phillip.allen@enron.com   
3       Mon, 23 Oct 2000 06:13: